<img 
    style="position: absolute; 
           left: 60%; 
           top: 0; /* Added to ensure proper positioning */
           height: 500px; 
           width: 40%; /* Maintain the original width */
           object-fit: cover; /* Adjust if necessary */
           clip-path: inset(0px 50px 0px 50px round 10px);" 
    src= "../.github/public_html/fig/Major crack Width development.png"
/>
</figure>


<h1 style="width: 60%; color: #EC6842; font-size: 55pt;">
    <Strong>
        FEA Study
    </Strong>
</h1>

<h2 id="Background"><B>
    Rationale for the project<a class="anchor-link" href="#Background">&#182;</a>
    </B>
</h2>
<p style="text-align: justify; width: 60%; font-weight: normal;">
     This studies case study is the building will also evaluate the capabilities of FEA for the assessment of unreinforced masonry structures undergoing subsidence. For this measure understanding and analysing the capabilities of a finite element analysis are a important part in the validation of the model and the presenting of results. This notebook illustrates some of the different tools built for the analysis of such models and its implementations in different analysis types. Due to the limitations of the analysis output characteristics from DIANA FEA not all components of these analysis are able to have been done algorithmically therefore, it is reccomended to read this modules README.MD notes in order to ensure the different requirements for these analysis are well implemented in your models. 
</p>

## <strong> X | Imports</strong>

In [1]:
from bricks.fea.analysis import analyse_models, setup_analysis
from bricks.fea.analysis.processing.tabulated import process_tb
from bricks.fea.analysis.main import *
from bricks.fea.analysis.processing.main import *
from bricks.fea.analysis.processing.utils import *
from bricks.fea.analysis.plots import *

import numpy as np
import matplotlib.pyplot as plt

## <strong> 2 | Process runs</strong>

### <strong> 2.1 | Run individual analysis</strong>

The three main individual plots for the relative displacement, crack width progression and damage level progression

In [2]:
# ------------------------------ Manual Analysis ----------------------------- #
analysis_info_TSRCM = {
    'Mutual': {
        'Node Nr': [[22, 23], [22, 23]],
        'Reference': [['TDtY', 'TDtY'], ['TDtY', 'TDtX']]
    },
    'Crack width': {
        'EOI': [[177,178,179,435],
                [35, 166, 203, 387, 523, 684, 723, 867],
                [9, 206, 263, 612]],
    },
    'Damage level': { 
        'parameters': {'auto': True,
                       'mesh': 200         
    }
}}

plot_settings_TSRCM = {
    'Mutual': {
        'traces': ['${u_{yB}}/{u_{yA}}$','${u_{xB}}/{u_{yA}}$ '],
        'labels': ['Displacement A [mm]', 'Displacement B [mm]'],
        'titles': 'Relative displacements at locations of interest',
        'scientific': False
    },
    'Crack width': {
        'traces': ['Crack 1','Crack 5','Crack 9'],
        'labels': ['Load factor $\lambda$', 'Crack Width $c_w$ [mm]'],
        'titles': 'Major crack Width development',
        'scientific': True
    },
    'Damage level': {
        'traces': ['$\psi$ TSCM'],
        'labels': ['Load factor $\lambda$', 'Damage Parameter $\psi$'],
        'titles': 'Damage level progression',
        'scientific': True
    }
}

merge_info = {
    'Titles': ['Force norm', 'Displacement norm'],
    'x_label': 'Load factor $\lambda$',
    'y_label': 'Disp \& Force Norm $|\Delta_f| |\Delta_u|$',
    'title': 'Combined Force and Displacement Norms'
}

dir = r'C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Modelling\Models\Continuum\Standard\W2O - TS'
analyse_models(dir,analysis_info_TSRCM, plot_settings_TSRCM, merge = merge_info)


# ---------------------------- Automatic analysis ---------------------------- #
analysis_info_mesh = {
    'Mutual': {
        'Node Nr': [[22, 23], [22, 23]],
        'Reference': [['TDtY', 'TDtY'], ['TDtY', 'TDtX']]
    },
    'Damage level': { 
        'parameters': {'auto': True,
                       'mesh': 200         
    }}
}

plot_settings_mesh = {
    'Mutual': {
        'traces': ['${u_{yB}}/{u_{yA}}$','${u_{xB}}/{u_{yA}}$'],
        'labels': ['Displacement A [mm]', 'Displacement B [mm]'],
        'titles': 'Relative displacements at locations of interest',
        'scientific': False
    },
    'Damage level': {
        'traces': ['$\psi$'],
        'labels': ['Load factor $\lambda$', 'Damage Parameter $\psi$'],
        'titles': 'Damage level progression',
        'scientific': True
    }
}

merge_info = {
    'Titles': ['Force norm', 'Displacement norm'],
    'x_label': 'Load factor $\lambda$',
    'y_label': 'Disp \& Force Norm $|\Delta_f| |\Delta_u|$',
    'title': 'Combined Force and Displacement Norms'
}

dir = r'C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Modelling\Models\Continuum\Standard\W2I - TS'
rr = analyse_models(dir,analysis_info_mesh, plot_settings_mesh, merge = merge_info)

{'Crack 0': {'length': 5062.054006033519, 'average_width': 7.876843971631206, 'component': 0, 'elements': [34.0, 35.0, 71.0, 88.0, 110.0, 146.0, 184.0, 211.0, 248.0, 259.0, 260.0, 273.0, 276.0, 293.0, 318.0, 335.0, 336.0, 363.0, 364.0, 367.0, 420.0, 552.0, 569.0, 607.0, 608.0, 609.0, 729.0, 731.0, 736.0, 750.0, 751.0, 794.0, 815.0, 850.0, 877.0, 887.0, 904.0, 966.0]}}


### <strong> 2.2 | Run compared analysis</strong>

The three main individual plots for the relative displacement, crack width progression and damage level progression between models

In [ ]:
# ------------------------------- TSRCM ------------------------------ #
combined_info_TSRCM = {
    'Mutual': {
        'Node Nr': [[22, 23], [22, 23]],
        'Reference': [['TDtY', 'TDtY'], ['TDtY', 'TDtX']]
    },
    'Damage level': { 
        'parameters': {'auto': True,
                       'mesh': 200         
    }
}}

combined_settings_TSRCM = {
    'Mutual': {
        'traces': ['${u_{yB}}/{u_{yA}}$ TSCM','${u_{xB}}/{u_{yA}}$ TSCM'],
        'labels': ['Displacement A [mm]', 'Displacement B [mm]'],
        'titles': 'Relative displacements at locations of interest',
        'scientific': False
    },
    'Damage level': {
        'traces': ['${\psi_{TSCM}}$ 200mm'],
        'labels': ['Load factor $\lambda$', 'Damage Parameter $\psi$'],
        'titles': 'Damage level progression',
        'scientific': True
    }
}


data_TSCM = {}
data_TSCM['name'] = 'TSCM-O',
data_TSCM['analysis_info'] = combined_info_TSRCM
data_TSCM['plot_settings'] = combined_settings_TSRCM
data_TSCM['dir'] = r'C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Modelling\Models\Continuum\Standard\W2O - TS\2DW2O_-_TS_NLA.tb'

combined_info_EMMS = {
    'Mutual': {
        'Node Nr': [[22, 23], [22, 23]],
        'Reference': [['TDtY', 'TDtY'], ['TDtY', 'TDtX']]
    },
    'Damage level': { 
            'parameters': {'auto': True,
                        'mesh': 200},
                         }}

combined_settings_EMMS = {
    'Mutual': {
        'traces': ['${u_{yB}}/{u_{yA}}$ EMM','${u_{xB}}/{u_{yA}}$ EMM'],
        'labels': ['Displacement A [mm]', 'Displacement B [mm]'],
        'titles': 'Relative displacements at locations of interest',
        'scientific': False
    },
    'Damage level': {
        'traces': ['${\psi_{EMM}}$'],
        'labels': ['Load factor $\lambda$', 'Damage Parameter $\psi$'],
        'titles': 'Damage level progression',
        'scientific': True
    }
}

data_EMMO = {}
data_EMMO['name'] = 'EMM - O'
data_EMMO['analysis_info'] = combined_info_EMMS
data_EMMO['plot_settings'] = combined_settings_EMMS
data_EMMO['dir'] = r'C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Modelling\Models\EMM\W2O - EMS\2DW2O_-_EMS_NLA.tb'

plot_data_list = [data_TSCM, data_EMMO]
cfigs = compare_models(plot_data_list)


### <strong> 2.3 | Script to take screenshots and videos of models</strong>

Creates a new directory in the models directory where it stores the contour plots for all the specified result components 

In [2]:
base_path = r''
config = {
    'results': [
        {
            'component': 'TDtY',
            'result': 'Displacements',
            'type': 'Node',
            'limits': [-35, -28, -24, -20, -16, -12, -8, -4, 0]
        },
        {
            'component': 'E1',
            'result': 'Total Strains',
            'type': 'Element',
            'location': 'mappedintpnt',
            'limits': [-0.004, -0.002, 0, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.02, 0.08]
        },
        {
            'component': 'S1',
            'result': 'Cauchy Total Stresses',
            'type': 'Element',
            'location': 'mappedintpnt',
            'limits': [-3.5, -2, -1, -0.05, -0.01, 0, 0.01, 0.05, 1, 3, 61]
        },
        {
            'component': 'Ecw1',
            'result': 'Crack-widths',
            'type': 'Element',
            'location': 'mappedintpnt',
            'limits': [0, 1, 2, 3, 4, 5, 10, 15, 20]
        }
    ],
    'script': {
        'analysis': "NLA",
        'load_cases': ['Building', 'Sub Deformation'],
        'load_steps': [30, 720],
        'load_factors_init': [0.0330000, 0.00138800],
        'snapshots': 6,
        'view_settings': {
            'view_point': [0, 0, 25.0, 0, 1, 0, 5.2, 3.1, 5.5e-17, 19, 3.25],
            'title_font_size': 36,
            'legend_font_size': 34,
            'annotation_font_size': 28
        }
    }
}

setup_analysis(base_path, config)

### <strong> 2.4 | Sensitivity effects on damage study</strong>

From all models the following code the influence of the damage and time against mesh factors

In [43]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

def plot_fits(ax, x, y, models, labels):
    vals = np.linspace(min(x), max(x), 100)
    for model, label in zip(models, labels):
        params, _ = curve_fit(model, x, y)
        ax.plot(vals, model(vals, *params), label=f"{label}: ${params[0]:.4f}x^{{{2 if label == 'Quadratic' else 1}}}$")
    ax.scatter(x, y, color='black', label='Observed Data', marker='s', facecolors='none', s=5, linewidths=0.5)
    ax.set_xlabel('Mesh Factor')
    ax.legend()

observed_values = np.array([132, 156, 0, 0, 528, 707, 161, 468, 1410, 2136, 1300, 1715]) - 0
mesh_factor = [1, 1, 1, 1, 2, 2, 2, 2, 4, 4, 4, 4]
percentage_diff_damage = np.array([42, 17, 17, 33, 25, 0, 13, 17, 17, 37, 0, 25])
mesh_factor_damage = [1, 2, 4] * 4

models = [
    lambda x, a, b: a * x + b,
    lambda x, a, b, c: a * x**2 + b * x + c,
    lambda x, a, b: b * np.power(x, a)
]

fig, axs = plt.subplots(1, 2, figsize=(8, 2.5))

plot_fits(axs[0], mesh_factor_damage, percentage_diff_damage, models, ['Linear', 'Quadratic', 'Power Law'])
axs[0].set_ylabel(r'DP percentage change $\Delta \% \Psi$')

plot_fits(axs[1], mesh_factor, observed_values, models, ['Linear', 'Quadratic', 'Power Law'])
axs[1].set_ylabel(r'Analysis time percentage change $\Delta \% t$')

plt.tight_layout()
plt.show()

## <strong> 3 | Calibration</strong>

### <strong> 3.1 | Feature Importance scores</strong>
From all the analysis the features are compared against the targets through a random forest to obtain the feature importance scores

In [16]:
from sklearn.ensemble import RandomForestRegressor
from math import pi
import pickle

def load_and_prepare_data(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    df = data["monitor_df"]
    targets_expanded = pd.DataFrame(df['Targets'].to_list(), columns=[f'Feature {i+1}' for i in range(len(df['Targets'][0]))])
    psi_expanded = pd.DataFrame(df['Psi'].to_list(), columns=['Psi'])
    df = pd.concat([df.drop(columns=['Targets', 'Psi', "Time", "Convergence"]), targets_expanded, psi_expanded], axis=1)
    
    return df

file_path_1 = r"C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Calibration\TSCM Single\!Results\Run_2024-08-29_20-07-43\wall_instance_latest.pkl"
features_1 = ["Young's Modulus $E_{y}$ [MPa]", "Tensile Strength $f_{t}$ [MPa]", "Mode I, Fracture Energy $G_{f-I}$ [N/mm]"]

file_path_2 = r"C:\Users\javie\OneDrive - Delft University of Technology\Year 2\Q3 & Q4\CIEM0500 - MS Thesis Project\!content\Experimentation\Calibration\EMM Single\!Results\Run_2024-09-01_09-41-38\wall_instance_latest.pkl"
features_2 = ["Vertical Young's Modulus $E_{y}$ [MPa]","Horizontal Young's Modulus $E_{x}$ [MPa]","Shear Modulus $G$ [MPa]", 
              "Tensile Strength $f_{t}$ [MPa]", "Mode I, Fracture Energy $G_{f-I}$ [N/mm]","Mode II, Fracture Energy $G_{f-II}$ [N/mm]"]

df1 = load_and_prepare_data(file_path_1)
df2 = load_and_prepare_data(file_path_2)

X1 = df1.drop(columns=['Total Loss', 'Psi'])
y1 = df1['Total Loss']

X2 = df2.drop(columns=['Total Loss', 'Psi'])
y2 = df2['Total Loss']

model1 = RandomForestRegressor()
model2 = RandomForestRegressor()

model1.fit(X1, y1)
model2.fit(X2, y2)

importance1 = model1.feature_importances_
importance2 = model2.feature_importances_

print("Feature Importances for df1:")
for feature, importance in zip(X1.columns, importance1):
    print(f'{feature}: {importance}')

print("\nFeature Importances for df2:")
for feature, importance in zip(X2.columns, importance2):
    print(f'{feature}: {importance}')

Feature Importances for df1:
Feature 1: 0.492916798644063
Feature 2: 0.3143455183565802
Feature 3: 0.19273768299935684

Feature Importances for df2:
Y_init: 0.0
X_init: 0.0
Feature 1: 0.18772326151437413
Feature 2: 0.1527343789029069
Feature 3: 0.14914064557695433
Feature 4: 0.18972323165041252
Feature 5: 0.13111651229497265
Feature 6: 0.1895619700603796


### <strong> 3.2 | Sensitivity Analysis</strong>

#### <strong> 3.2.1 | Plotting the GP solution space</strong>

In [ ]:
import pickle
import torch
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.priors import GammaPrior
from gpytorch.constraints import Interval
from scipy.stats import norm
from matplotlib.ticker import FormatStrFormatter

pickle_file_path = '/content/drive/MyDrive/CALIBRATION/wall_instance_latest_TSCM.pkl'
with open(pickle_file_path, 'rb') as f:
    saved_data_tscm = pickle.load(f)

material_params = MaterialParameters()
confidence_level = 95
Ey_bounds = material_params.get_bounds('Emy', confidence_level)
fw_bounds = material_params.get_bounds('fw', confidence_level)
tensile_strength_bounds = tuple([material_params.tensile_strength(value) for value in fw_bounds])
tensile_fracture_energy_bounds = tuple([material_params.tensile_fracture_energy(value, 'mortar') for value in fw_bounds])
bounds = [Ey_bounds, tensile_strength_bounds, tensile_fracture_energy_bounds]

filtered_df_tscm = saved_data_tscm["monitor_df"][saved_data_tscm["monitor_df"]["Psi"].apply(lambda x: x[0] < 7)]

X_reco = np.vstack(filtered_df_tscm["Targets"])
X_scaler = MinMaxScaler()
X_init_single_scaled = X_scaler.fit_transform(X_reco)

Y_psi = np.array([i[0] for i in filtered_df_tscm["Psi"].values]).reshape(-1, 1)
Y_scaler = MinMaxScaler()
Y_psi_scaled = Y_scaler.fit_transform(Y_psi)

min_psi_index = np.argmin(Y_psi)
optimal_params = X_reco[min_psi_index]

likelihood = GaussianLikelihood(noise_constraint=Interval(1e-5, 6.8e-3))  # Regularize
kernel = ScaleKernel(
    RBFKernel(ard_num_dims=len(bounds), lengthscale_prior=GammaPrior(3.0, 0.5)) +
    MaternKernel(nu=2.5, ard_num_dims=len(bounds), lengthscale_prior=GammaPrior(3.0, 0.5))
)

gp_model_tscm = SingleTaskGP(
    train_X=torch.tensor(X_init_single_scaled, dtype=torch.float64),
    train_Y=torch.tensor(Y_psi_scaled, dtype=torch.float64),
    covar_module=kernel,
    likelihood=likelihood
)

mll = ExactMarginalLogLikelihood(gp_model_tscm.likelihood, gp_model_tscm)
fit_gpytorch_mll(mll)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_vars = X_init_single_scaled.shape[1]
grid_size = 50  # Adjust based on computational resources

# Create linspace values for each parameter based on its bounds
linspace_values = [
    np.linspace(bounds[i][0], bounds[i][1], grid_size)
    for i in range(num_vars)
]

In [ ]:
# ---------------------------------------------------------------------------- #
#                               Plotting the Mean                              #
# ---------------------------------------------------------------------------- #
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import torch
from sklearn.preprocessing import StandardScaler

num_vars = 3
fig, axes = plt.subplots(num_vars, num_vars, figsize=(12, 10))
fig.subplots_adjust(wspace=0.3, hspace=0.2)
labels = ["Young's Modulus $E$ [MPa]", "Tensile Strength $f_{t}$ [MPa]", "Mode I, Fracture Energy $G_{f-I}$ [N/mm]"]

global_min, global_max = float('inf'), float('-inf')

for i in range(num_vars):
    for j in range(i + 1, num_vars):
        X1, X2 = np.meshgrid(linspace_values[i], linspace_values[j])
        input_grid = np.zeros((X1.size, num_vars))
        input_grid[:, i] = X1.ravel()
        input_grid[:, j] = X2.ravel()
        for k in range(num_vars):
            if k != i and k != j:
                input_grid[:, k] = optimal_params[k]

        input_grid_normalized = X_scaler.transform(input_grid)
        input_grid_normalized = torch.tensor(input_grid_normalized, device=device, dtype=torch.double)
        with torch.no_grad():
            posterior = gp_model_tscm.posterior(input_grid_normalized)
            mean_scaled = posterior.mean.cpu().numpy().reshape(X1.shape)

        mean_unscaled = Y_scaler.inverse_transform(mean_scaled.reshape(-1, 1)).reshape(X1.shape)
        global_min = min(global_min, mean_unscaled.min())
        global_max = max(global_max, mean_unscaled.max())

global_min = np.floor(global_min * 100) / 100
global_max = np.ceil(global_max * 100) / 100

levels = np.linspace(global_min, global_max, 10)  # Change the number of levels as needed
levels = np.round(levels, 2)  # Round levels to two decimal places
cmap = plt.get_cmap('plasma', len(levels)-1)  # Get a colormap with enough colors for your levels
norm = colors.BoundaryNorm(levels, ncolors=cmap.N, clip=True)

for i in range(num_vars):
    for j in range(num_vars):
        ax = axes[i, j]
        if i == j:
            X1 = linspace_values[i]
            input_grid = np.tile(optimal_params, (len(X1), 1))
            input_grid[:, i] = X1

            input_grid_normalized = X_scaler.transform(input_grid)
            input_grid_normalized = torch.tensor(input_grid_normalized, device=device, dtype=torch.double)
            with torch.no_grad():
                posterior = gp_model_tscm.posterior(input_grid_normalized)
                mean_scaled = posterior.mean.cpu().numpy()
                std_scaled = posterior.variance.sqrt().cpu().numpy()

            mean_unscaled = Y_scaler.inverse_transform(mean_scaled.reshape(-1, 1)).reshape(-1)

            ax.plot(X1, mean_unscaled, color='black')
            ax.set_xlabel(labels[i], labelpad=5)
            ax.set_ylabel('Damage Parameter ($\psi$)', labelpad=5)
        else:
            X1, X2 = np.meshgrid(linspace_values[i], linspace_values[j])
            input_grid = np.zeros((X1.size, num_vars))
            input_grid[:, i] = X1.ravel()
            input_grid[:, j] = X2.ravel()
            for k in range(num_vars):
                if k != i and k != j:
                    input_grid[:, k] = optimal_params[k]

            input_grid_normalized = X_scaler.transform(input_grid)
            input_grid_normalized = torch.tensor(input_grid_normalized, device=device, dtype=torch.double)
            with torch.no_grad():
                posterior = gp_model_tscm.posterior(input_grid_normalized)
                mean_scaled = posterior.mean.cpu().numpy().reshape(X1.shape)

            mean_unscaled = Y_scaler.inverse_transform(mean_scaled.reshape(-1, 1)).reshape(X1.shape)
            cont = ax.contourf(X2, X1, mean_unscaled, levels=levels, cmap=cmap, norm=norm, extend='both')
            ax.set_xlabel(labels[j], labelpad=5)
            ax.set_ylabel(labels[i], labelpad=5)

cbar = fig.colorbar(cont, ax=axes.ravel().tolist(), orientation='vertical', shrink=0.8, aspect=20, extend='both', pad=0.025)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cbar.set_label('Expected Damage Parameter ($\psi$) \n **in region close to the optimal solution', rotation=270, labelpad=20)

plt.show()
# ---------------------------------------------------------------------------- #
#                             Plotting the variance                            #
# ---------------------------------------------------------------------------- #
import numpy as np
import matplotlib.pyplot as plt
import torch
from sklearn.preprocessing import StandardScaler

# Assuming other necessary setups like model, linspace_values, optimal_params, device are defined above.

num_vars = 3
fig, axes = plt.subplots(num_vars, num_vars, figsize=(12, 10))
fig.subplots_adjust(wspace=0.3, hspace=0.2)
labels = ["Young's Modulus $E$ [MPa]", "Tensile Strength $f_{t}$ [MPa]", "Mode I, Fracture Energy $G_{f-I}$ [N/mm]"]

# Loop through each pair of variables to simulate and find the standard deviation
for i in range(num_vars):
    for j in range(num_vars):
        ax = axes[i, j]
        if i == j:
            # Plot the diagonal terms: mean and uncertainty bounds
            X1 = linspace_values[i]
            input_grid = np.tile(optimal_params, (len(X1), 1))
            input_grid[:, i] = X1

            input_grid_normalized = X_scaler.transform(input_grid)
            input_grid_normalized = torch.tensor(input_grid_normalized, device=device, dtype=torch.double)
            with torch.no_grad():
                posterior = gp_model_tscm.posterior(input_grid_normalized)
                variance = posterior.variance.cpu().numpy().reshape(X1.shape)  # Use raw variance data

                # Calculate standard deviation in the scaled space
                std_scaled = np.sqrt(variance)

                # Inverse transform the standard deviation to the original scale
                std_unscaled = Y_scaler.inverse_transform(std_scaled.reshape(-1, 1)).reshape(X1.shape)

            ax.plot(X1, std_unscaled, color='black')
            ax.set_xlabel(labels[i], labelpad=5)
            ax.set_ylabel('Damage Parameter ($\psi$)', labelpad=5)
        else:
            X1, X2 = np.meshgrid(linspace_values[i], linspace_values[j])
            input_grid = np.zeros((X1.size, num_vars))
            input_grid[:, i] = X1.ravel()
            input_grid[:, j] = X2.ravel()
            for k in range(num_vars):
                if k != i and k != j:
                    input_grid[:, k] = optimal_params[k]

            input_grid_normalized = X_scaler.transform(input_grid)
            input_grid_normalized = torch.tensor(input_grid_normalized, device=device, dtype=torch.double)
            with torch.no_grad():
                posterior = gp_model_tscm.posterior(input_grid_normalized)
                variance = posterior.variance.cpu().numpy().reshape(X1.shape)  # Use raw variance data

                # Calculate standard deviation in the scaled space
                std_scaled = np.sqrt(variance)

                # Inverse transform the standard deviation to the original scale
                std_unscaled = Y_scaler.inverse_transform(std_scaled.reshape(-1, 1)).reshape(X1.shape)

            # Plot the unscaled standard deviation
            cont = ax.contourf(X2, X1, std_unscaled, cmap='coolwarm', extend='both')
            ax.set_xlabel(labels[j], labelpad=5)
            ax.set_ylabel(labels[i], labelpad=5)

# Create the colorbar
cbar = fig.colorbar(cont, ax=axes.ravel().tolist(), orientation='vertical', shrink=0.8, aspect=20, extend='both', pad=0.025)
# Set the colorbar to display values with two decimal points
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cbar.set_label('Damage Parameter ($\psi$) Standard Deviation \n **in region close to the optimal solution', rotation=270, labelpad=20)
plt.show()

#### <strong> 3.2.2 | Plotting the Kernel, Covariance between variables</strong>

In [ ]:
grid_size = 20
colormap = 'coolwarm'
figsize = (10, 8)
num_vars = len(bounds)
labels = ["Young's Modulus $E$ [MPa]", "Tensile Strength $f_{t}$ [MPa]", "Mode I, Fracture Energy $G_{f-I}$ [N/mm]"]
gp_model = gp_model_tscm

fig, axes = plt.subplots(num_vars, num_vars, figsize=figsize)
fig.subplots_adjust(wspace=0.35, hspace=0.3)

def compute_covariance(i, j):
    X1 = np.linspace(bounds[i][0], bounds[i][1], grid_size)
    X2 = np.linspace(bounds[j][0], bounds[j][1], grid_size)

    set1 = np.tile(optimal_params, (grid_size, 1))
    set2 = np.tile(optimal_params, (grid_size, 1))

    set1[:, i] = X1  
    set2[:, j] = X2  

    # Normalize the arrays
    set1_normalized = X_scaler.transform(set1)
    set2_normalized = X_scaler.transform(set2)

    set1_normalized = torch.tensor(set1_normalized, device=device, dtype=torch.double)
    set2_normalized = torch.tensor(set2_normalized, device=device, dtype=torch.double)

    with torch.no_grad():
        cov_matrix = gp_model.covar_module(set1_normalized, set2_normalized).evaluate().cpu().numpy()

    cov_matrix = cov_matrix.reshape(grid_size, grid_size)
    cov_matrix = Y_scaler.inverse_transform(cov_matrix.reshape(-1, 1)).reshape(grid_size, grid_size)

    return X1, X2, cov_matrix

all_covariances = []
for i in range(num_vars):
    for j in range(num_vars):
        X1, X2, cov_matrix = compute_covariance(i, j)
        all_covariances.append(cov_matrix)

all_covariances_flat = np.concatenate([cov.flatten() for cov in all_covariances])
mean_covariance = np.mean(all_covariances_flat)

vmin = np.min(all_covariances_flat - mean_covariance)
vmax = np.max(all_covariances_flat - mean_covariance)

for i in range(num_vars):
    for j in range(num_vars):
        ax = axes[i, j]
        X1, X2, cov_matrix = compute_covariance(i, j)

        cov_matrix_deviation = cov_matrix - mean_covariance

        cont = ax.contourf(X2, X1, cov_matrix_deviation, cmap=colormap, vmin=vmin, vmax=vmax, levels=10, extend='both')

        ax.set_xlabel(labels[j], labelpad=5)
        ax.set_ylabel(labels[i], labelpad=5)

cbar = fig.colorbar(cont, ax=axes.ravel().tolist(), orientation='vertical', shrink=0.8, aspect=20, extend='both', pad=0.025)

cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.1e'))
cbar.set_label(f'Model parameters covariance ($\\mu$ = {mean_covariance:.3f})\n**in region close to the optimal solution', rotation=270, labelpad=20)

plt.show()